
     
# Implement the ridge regression algorithm

   - Loss function
$$L(\mathbf{w}, b) =\frac{1}{n}\sum_{i=1}^n l^{(i)}(\mathbf{w}, b) =\frac{1}{n} \sum_{i=1}^n \frac{1}{2}\left(\mathbf{w}^\top \mathbf{x}^{(i)} + b - y^{(i)}\right)^2 + \lambda \mathbf{w}^\top\mathbf{w}.$$
  - The first term is the least square error loss, which is already defined in the codes for linear regression in Chapter 3
  - The second term is the sqaured norm of the weight vector $\mathbf{w}$. For example, if $\mathbf{w}=[1,2,3]$, then $\mathbf{w}^T\mathbf{w}=1^2+2^2+3^2=14$. For more about norms and dot product of vectors, please read the slides of Lecture 1 regarding linear algebra.

In [1]:
%matplotlib inline
import random
import torch
from d2l import torch as d2l

In [2]:
def synthetic_data(w, b, num_examples):  #@save
    """Generate y = Xw + b + noise."""
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

### Parameters



In [3]:
data_dim=100
data_size=100
batch_size=20
lr = 0.03
num_epochs = 10
lambd = 0.01;

## Geneate the true linear model and the data

In [4]:
#true_w_o = torch.tensor([2, -3.4])
torch.manual_seed(0)
true_w = torch.normal(0,1,(1,data_dim))
true_w = true_w.reshape([data_dim])
true_b = 4.2
#true_b = 0
features, labels = synthetic_data(true_w, true_b, data_size)


#print('features:', features[0],'\nlabel:', labels[0])

## Split the data into multiple subsets (batches)

In [5]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # The examples are read at random, in no particular order
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(
            indices[i: min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]

In [6]:
#batch_size = 10
#batch_size = 20
#for X, y in data_iter(batch_size, features, labels):
#    print(X, '\n', y)
#    break

## Initialization of the weight vector and the bias 

In [7]:
w = torch.normal(0, 0.01, size=(data_dim,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

## Define the linear model

In [8]:
def linreg(X, w, b):  #@save
    """The linear regression model."""
    return torch.matmul(X, w) + b

## Define the squared loss and the $L_2$ penalty

In [9]:
def squared_loss(y_hat, y):  #@save
    """Squared loss."""
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

def l2_penalty(w):
    return torch.sum(w.pow(2)) / 2

## Define the optimization algorithm: stochastic gradient descent (sgd) method

In [10]:
def sgd(params, lr, batch_size):  #@save
    """Minibatch stochastic gradient descent."""
    with torch.no_grad():
        for param in params:
            param -= lr * (param.grad / batch_size)
            param.grad.zero_()

## Set linreg as the net (the model object in neural network implementations)

In [11]:
#trainer = torch.optim.SGD([w,b], lr=0.03, weight_decay=0.01)
#lr = 0.03
#num_epochs = 1000
net = linreg
loss = squared_loss
#loss = ridge_loss
#lambd = 0.1;

## Training
   - Each epoch goes through all the batches (which means the entire training set)
   - Overall loss $l$: sum of the squared loss and the penalty term
   - l.sum().backward: compute the gradients of the loss function with respect to the model parameters ($w$ and $b$ for linear regression). You can think the negative of the gradient as the best direction to reduce the loss function. 

In [12]:
for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        #l = loss(net(X, w, b), y)  # Minibatch loss in `X` and `y`
        l = loss(net(X, w , b), y) + lambd * l2_penalty(w)
        # Compute gradient on `l` with respect to [`w`, `b`]
        l.sum().backward()
        sgd([w, b], lr, batch_size)  # Update parameters using their gradient
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        # train_l = loss(net(features, w, b), labels,w)
        #print(train_l.shape)
        
        print(f'epoch {epoch + 1}, {float(train_l.mean()):f}')

epoch 1, 40.005669
epoch 2, 24.424444
epoch 3, 15.807398
epoch 4, 10.741873
epoch 5, 7.665995
epoch 6, 5.731526
epoch 7, 4.405257
epoch 8, 3.502190
epoch 9, 2.854735
epoch 10, 2.373071


In [13]:
#print(f'error in estimating w: {true_w - w.reshape(true_w.shape)}')
print(f'error in estimating w: {torch.linalg.norm(true_w - w.reshape(true_w.shape))}')
print(f'error in estimating b: {true_b - b}')

error in estimating w: 4.98659086227417
error in estimating b: tensor([1.5187], grad_fn=<RsubBackward1>)
